In [1]:
import urllib
import requests
import os
from copy import deepcopy as mcopy
from music21 import *

In [9]:
# Initialize Music21 and 

defaults.author = ''
defaults.title = ''

def send_m21_object(obj, scoreName):
    xml = musicxml.m21ToString.fromMusic21Object(obj)
    headers = {'Content-Type': 'text/xml'}
    url = urllib.parse.urljoin('http://ks-images:3000/score/', scoreName)
    return requests.post(url, 
                        data=xml,
                        headers=headers,
                        timeout=5)
def write_midi_file(a_stream, filename='default.mid'):
    mf = midi.translate.streamToMidiFile(a_stream)
    mf.open(filename, 'wb')
    mf.write()
    mf.close()

In [10]:
def append_octave_up(stream):
    """Append a copy of the stream an octave up (in place)"""
    for n in stream.notes:
        stream.append(n.transpose(12))
    return stream

def all_but_last(a_stream):
    """Return a new stream that includes all but the last note.
    Discard any non-note items in the stream.
    """
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    if len(notes) <= 1:
        return result
    result.append(notes[:-1])
    return result

def all_but_first_and_last(a_stream):
    """Return a new stream of the same type that includes all but
    the first and last rests/notes"""
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    if len(notes) <= 2:
        return result
    result.append(notes[1:-1])
    return result

def reverse_notes(a_stream):
    """Return a new stream with the notes and rests reversed."""
    Class = a_stream.__class__
    result = Class()
    notes = list(mcopy(a_stream).notesAndRests)
    notes.reverse()
    result.append(notes)
    return result

def all_but_first(a_stream):
    """"""
    return reverse_notes(all_but_last(reverse_notes(a_stream)))

def arp_forward_rev_x_times(a_stream, x):
    """Arpeggiate a stream of notes x times. Playback the stream
    in reverse order, without playing the last note in the
    original stream twice. Finish on the note where we started.
    go through and back x times."""
    
    Class = a_stream.__class__
    result = Class()

    # All but last reversed
    ablr = reverse_notes(all_but_last(a_stream))
    result.append(a_stream)
    result.append(ablr)

    appendage = all_but_first(mcopy(result).flat)    
    result.repeatAppend(appendage, x - 1)
    return result.flat


In [11]:
def hashtag_music(note_list):
    s1 = stream.Part()
    noteList = [note.Note(n, type='16th') for n in note_list]
    s1.append(noteList)
    append_octave_up(s1)
    return arp_forward_rev_x_times(s1, 4)

In [24]:
n1 = ['C3', 'D3', 'G3', 'A3']
stream1 = hashtag_music(n1)
stream1.insert(clef.BassClef())
send_m21_object(stream1, 'a')

n2 = ['C3', 'e3', 'G3', 'b3']
stream2 = hashtag_music(n2)
stream2.insert(clef.BassClef())
send_m21_object(stream2, 'a')

n3 = ['D3', 'E3', 'A3', 'B3']
stream3 = hashtag_music(n3)
stream3.insert(clef.BassClef())
send_m21_object(stream3, 'a')

<Response [200]>

In [ ]:
c1 = 

In [21]:
write_midi_file(stream2)

In [19]:
write_midi_file(stream1)

In [26]:
write_midi_file(stream3)

In [34]:
three_sections = stream.Stream()
three_sections.append([stream1, stream2, stream3])
write_midi_file(three_sections.flat)

In [35]:
three_sections

<music21.stream.Stream 0x7f7cf1212860>